In [1]:
# ysk data cleaning part 1
import pandas as pd
import fuzzywuzzy as fz
from fuzzywuzzy import process

C:\Users\boran\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
path = 'C:/Users/boran/Desktop/Data Cleaning/'
filename = 'Dataset.xlsx'

In [3]:
df = pd.read_excel(path+filename)
df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Sandık No,Kayıtlı Seçmen Sayısı,Oy Kullanan Seçmen Sayısı,İtirazsız Geçerli Oy Sayısı,İtirazlı Geçerli Oy Sayısı,Toplam Geçerli Oy,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,1001,369,286,277,0,277,...,1,109,35,1.0,4,0,3,0,0,0
1,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,1002,368,316,307,0,307,...,0,116,34,0.0,3,0,1,0,1,0
2,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,1003,368,329,323,0,323,...,1,122,43,1.0,6,0,1,0,0,0
3,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,1004,368,321,317,0,317,...,0,115,42,0.0,17,0,0,0,3,0
4,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,1005,368,348,343,0,343,...,0,124,50,0.0,8,0,2,0,0,0


In [4]:
# Gereksiz sütunları dropla
df.drop(['Sandık No','Kayıtlı Seçmen Sayısı','İtirazsız Geçerli Oy Sayısı',
         'İtirazlı Geçerli Oy Sayısı','Toplam Geçerli Oy','Toplam Geçersiz Oy'],
        axis= 'columns',
        inplace = True)
df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,286,0,1,0,0,0,...,1,109,35,1.0,4,0,3,0,0,0
1,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,316,1,0,1,0,0,...,0,116,34,0.0,3,0,1,0,1,0
2,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,329,0,0,0,1,0,...,1,122,43,1.0,6,0,1,0,0,0
3,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,321,0,0,0,0,0,...,0,115,42,0.0,17,0,0,0,3,0
4,1,İSTANBUL,ÜSKÜDAR,AHMEDİYE MAH.,348,0,0,0,0,0,...,0,124,50,0.0,8,0,2,0,0,0


In [5]:
# Veri sandık seviyesinde. Bunu mahalle seviyesine dönüştürüyoruz.
aggregated_df = df.groupby(['Bölge','İl Adı', 'İlçe Adı', 'Mahalle/Köy']).sum()

# Reset index if you want these columns back as regular columns
aggregated_df = aggregated_df.reset_index()

# Now, 'aggregated_df' contains the sum of votes for each party, grouped by 'İl Adı', 'İlçe Adı', and 'Mahalle/Köy'
aggregated_df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,İSTANBUL,ADALAR,BURGAZADA MAH.,1255,1,0,2,3,5,...,4,704,70,1.0,12,0,2,1,9,0
1,1,İSTANBUL,ADALAR,HEYBELİADA MAH.,2912,5,2,8,1,4,...,2,1203,307,3.0,43,0,5,0,5,0
2,1,İSTANBUL,ADALAR,KINALIADA MAH.,1528,0,2,3,1,4,...,0,724,92,3.0,15,0,4,0,4,0
3,1,İSTANBUL,ADALAR,MADEN MAH.,3438,1,1,4,4,13,...,3,1469,228,4.0,62,0,8,1,12,0
4,1,İSTANBUL,ADALAR,NİZAM MAH.,2702,2,0,7,3,7,...,5,1132,211,2.0,45,0,8,2,12,0


In [6]:
# Her partinin yüzde kaç oy aldığını hesaplıyoruz.
for column in aggregated_df.iloc[:,5:]:
    print('Column Name : ', column)
    aggregated_df[column] = 100*(aggregated_df[column]/aggregated_df['Oy Kullanan Seçmen Sayısı'])
    
aggregated_df.head()

Column Name :  MİLLET
Column Name :  HAK-PAR
Column Name :  TKP
Column Name :  TKH
Column Name :  SOL PARTİ
Column Name :  GENÇPARTİ
Column Name :  MEMLEKET
Column Name :  BBP
Column Name :  AK PARTİ
Column Name :  YENİDEN REFAH
Column Name :  MHP
Column Name :  YEŞİL SOL PARTİ
Column Name :  TİP
Column Name :  AB
Column Name :  ANAP
Column Name :  YP
Column Name :  HKP
Column Name :  MİLLİ YOL
Column Name :  VATAN PARTİSİ
Column Name :  GBP
Column Name :  CHP
Column Name :  İYİ PARTİ
Column Name :  AP
Column Name :  ZAFER PARTİSİ
Column Name :  SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI
Column Name :  CUMHUR İTTİFAKI
Column Name :  EMEK VE ÖZGÜRLÜK İTTİFAKI
Column Name :  MİLLET İTTİFAKI
Column Name :  ATA İTTİFAKI


,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,İSTANBUL,ADALAR,BURGAZADA MAH.,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.318725,56.095618,5.577689,0.079681,0.956175,0.0,0.159363,0.079681,0.717131,0.0
1,1,İSTANBUL,ADALAR,HEYBELİADA MAH.,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.068681,41.311813,10.542582,0.103022,1.476648,0.0,0.171703,0.000000,0.171703,0.0
2,1,İSTANBUL,ADALAR,KINALIADA MAH.,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,47.382199,6.020942,0.196335,0.981675,0.0,0.261780,0.000000,0.261780,0.0
3,1,İSTANBUL,ADALAR,MADEN MAH.,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.087260,42.728330,6.631763,0.116347,1.803374,0.0,0.232693,0.029087,0.349040,0.0
4,1,İSTANBUL,ADALAR,NİZAM MAH.,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.185048,41.894893,7.809030,0.074019,1.665433,0.0,0.296077,0.074019,0.444115,0.0


In [7]:
# Encoding problemini halletmemiz lazım.
'İstanbul'.lower()

'i̇stanbul'

In [8]:
""" 
Bu fonksiyon string sütunlarda İ'yi I'ya dönüştürüyor çünkü lower() metodu İ'yi i yapmıyor i̇ yapıyor.
"""

def replace_character(df):
    for column in df.columns:  # Changed aggregated_df to df
        if df[column].dtype == 'object':
            df[column] = df[column].str.replace('İ', 'I')
    return df
aggregated_df = replace_character(aggregated_df)

aggregated_df.head()


,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,ISTANBUL,ADALAR,BURGAZADA MAH.,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.318725,56.095618,5.577689,0.079681,0.956175,0.0,0.159363,0.079681,0.717131,0.0
1,1,ISTANBUL,ADALAR,HEYBELIADA MAH.,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.068681,41.311813,10.542582,0.103022,1.476648,0.0,0.171703,0.000000,0.171703,0.0
2,1,ISTANBUL,ADALAR,KINALIADA MAH.,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,47.382199,6.020942,0.196335,0.981675,0.0,0.261780,0.000000,0.261780,0.0
3,1,ISTANBUL,ADALAR,MADEN MAH.,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.087260,42.728330,6.631763,0.116347,1.803374,0.0,0.232693,0.029087,0.349040,0.0
4,1,ISTANBUL,ADALAR,NIZAM MAH.,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.185048,41.894893,7.809030,0.074019,1.665433,0.0,0.296077,0.074019,0.444115,0.0


In [9]:
# Tüm string sütunları lowercase yapıyoruz.
def lowercase_strings(x):
    if x.dtype == "object":  # Checking if the column is of string type
        return x.str.lower()
    return x

# Apply the function across the DataFrame
lowered_df = aggregated_df.apply(lowercase_strings)

lowered_df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,istanbul,adalar,burgazada mah.,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.318725,56.095618,5.577689,0.079681,0.956175,0.0,0.159363,0.079681,0.717131,0.0
1,1,istanbul,adalar,heybeliada mah.,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.068681,41.311813,10.542582,0.103022,1.476648,0.0,0.171703,0.000000,0.171703,0.0
2,1,istanbul,adalar,kinaliada mah.,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,47.382199,6.020942,0.196335,0.981675,0.0,0.261780,0.000000,0.261780,0.0
3,1,istanbul,adalar,maden mah.,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.087260,42.728330,6.631763,0.116347,1.803374,0.0,0.232693,0.029087,0.349040,0.0
4,1,istanbul,adalar,nizam mah.,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.185048,41.894893,7.809030,0.074019,1.665433,0.0,0.296077,0.074019,0.444115,0.0


In [10]:

# Karakterler ile ilgili bir problem kalmış mı diye test ediyoruz. 
# String sütunlardaki tüm distinct karakterleri yazdırıyor.
def find_and_sort_unique_characters(df):
    unique_chars = set()
    unique_chars_lower = set()
    for column in df.columns:
        if df[column].dtype == 'object':
            for item in df[column]:
                unique_chars.update(set(item))
                unique_chars_lower.update(set(char.lower() for char in item))

    # Sort the sets
    sorted_chars = sorted(unique_chars)
    sorted_chars_lower = sorted(unique_chars_lower)

    return sorted_chars, sorted_chars_lower

# Get and sort unique characters
sorted_unique_characters, sorted_unique_characters_lower = find_and_sort_unique_characters(lowered_df)

# Print or process the sorted unique characters
print("Original Characters:", sorted_unique_characters)
print("Lowercased Characters:", sorted_unique_characters_lower)

Original Characters: [' ', '(', ')', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ç', 'ö', 'ü', 'ğ', 'ş']
Lowercased Characters: [' ', '(', ')', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ç', 'ö', 'ü', 'ğ', 'ş']


In [11]:
lowered_df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,istanbul,adalar,burgazada mah.,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.318725,56.095618,5.577689,0.079681,0.956175,0.0,0.159363,0.079681,0.717131,0.0
1,1,istanbul,adalar,heybeliada mah.,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.068681,41.311813,10.542582,0.103022,1.476648,0.0,0.171703,0.000000,0.171703,0.0
2,1,istanbul,adalar,kinaliada mah.,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,47.382199,6.020942,0.196335,0.981675,0.0,0.261780,0.000000,0.261780,0.0
3,1,istanbul,adalar,maden mah.,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.087260,42.728330,6.631763,0.116347,1.803374,0.0,0.232693,0.029087,0.349040,0.0
4,1,istanbul,adalar,nizam mah.,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.185048,41.894893,7.809030,0.074019,1.665433,0.0,0.296077,0.074019,0.444115,0.0


In [12]:
ref_filename = 'ilçe_mahalle_geo_.xlsx'
referance_df = pd.read_excel(path+ref_filename)
referance_df.head()

,geometry,mahalle_corrdinate,mahalle_id,ilçe_id,ilçe,mahalle
0,Polygon,"29.055564,40.9087843,29.0555266,40.9089452,29....",relation/9460393,relation/963209,Adalar,Kınalıada Mahallesi
1,MultiPolygon,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",relation/9460394,relation/963209,Adalar,Burgazada Mahallesi
2,MultiPolygon,"29.0832149,40.8768968,29.082399,40.876799,29.0...",relation/9460395,relation/963209,Adalar,Heybeliada Mahallesi
3,MultiPolygon,"29.121725,40.8521407,29.1215908,40.8526682,29....",relation/9460396,relation/963209,Adalar,Nizam Mahallesi
4,MultiPolygon,"29.121725,40.8521407,29.1217464,40.8519378,29....",relation/9460397,relation/963209,Adalar,Maden Mahallesi


In [13]:
referance_df = replace_character(referance_df)
lowered_ref = referance_df.apply(lowercase_strings)
lowered_ref.head()


,geometry,mahalle_corrdinate,mahalle_id,ilçe_id,ilçe,mahalle
0,polygon,"29.055564,40.9087843,29.0555266,40.9089452,29....",relation/9460393,relation/963209,adalar,kınalıada mahallesi
1,multipolygon,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",relation/9460394,relation/963209,adalar,burgazada mahallesi
2,multipolygon,"29.0832149,40.8768968,29.082399,40.876799,29.0...",relation/9460395,relation/963209,adalar,heybeliada mahallesi
3,multipolygon,"29.121725,40.8521407,29.1215908,40.8526682,29....",relation/9460396,relation/963209,adalar,nizam mahallesi
4,multipolygon,"29.121725,40.8521407,29.1217464,40.8519378,29....",relation/9460397,relation/963209,adalar,maden mahallesi


In [14]:
def remove_unwanted_substring(df_column, unwanted_substring):
    """
    Removes occurrences of an unwanted substring from a DataFrame column.

    Parameters:
    df_column (pd.Series): The DataFrame column from which to remove the substring.
    unwanted_substring (str): The substring to remove.

    Returns:
    pd.Series: The modified column with the unwanted substring removed.
    """
    # Use the str accessor with replace to handle substring replacement
    return df_column.str.replace(unwanted_substring, '', regex=True)

# Assuming lowered_ref is your dataframe and 'mahalle' is the column to be corrected
lowered_ref['mahalle'] = remove_unwanted_substring(lowered_ref['mahalle'], ' mahallesi')
lowered_ref.head()

,geometry,mahalle_corrdinate,mahalle_id,ilçe_id,ilçe,mahalle
0,polygon,"29.055564,40.9087843,29.0555266,40.9089452,29....",relation/9460393,relation/963209,adalar,kınalıada
1,multipolygon,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",relation/9460394,relation/963209,adalar,burgazada
2,multipolygon,"29.0832149,40.8768968,29.082399,40.876799,29.0...",relation/9460395,relation/963209,adalar,heybeliada
3,multipolygon,"29.121725,40.8521407,29.1215908,40.8526682,29....",relation/9460396,relation/963209,adalar,nizam
4,multipolygon,"29.121725,40.8521407,29.1217464,40.8519378,29....",relation/9460397,relation/963209,adalar,maden


In [15]:
# Geometry sütunu oluşturduk
from shapely.geometry import Polygon

lowered_ref['mahalle_corrdinate'] = lowered_ref['mahalle_corrdinate'].astype(str)

# Define a function to convert the coordinate string to a Polygon object
def create_polygon(coord_string):
    coords = [float(coord) for coord in coord_string.split(',')]
    coord_pairs = list(zip(coords[::2], coords[1::2]))
    return Polygon(coord_pairs)

lowered_ref['geometry_string'] = lowered_ref['mahalle_corrdinate'].apply(create_polygon)

lowered_ref.head()

,geometry,mahalle_corrdinate,mahalle_id,ilçe_id,ilçe,mahalle,geometry_string
0,polygon,"29.055564,40.9087843,29.0555266,40.9089452,29....",relation/9460393,relation/963209,adalar,kınalıada,"POLYGON ((29.055564 40.9087843, 29.0555266 40...."
1,multipolygon,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",relation/9460394,relation/963209,adalar,burgazada,"POLYGON ((29.0574181 40.8840861, 29.05749 40.8..."
2,multipolygon,"29.0832149,40.8768968,29.082399,40.876799,29.0...",relation/9460395,relation/963209,adalar,heybeliada,"POLYGON ((29.0832149 40.8768968, 29.082399 40...."
3,multipolygon,"29.121725,40.8521407,29.1215908,40.8526682,29....",relation/9460396,relation/963209,adalar,nizam,"POLYGON ((29.121725 40.8521407, 29.1215908 40...."
4,multipolygon,"29.121725,40.8521407,29.1217464,40.8519378,29....",relation/9460397,relation/963209,adalar,maden,"POLYGON ((29.121725 40.8521407, 29.1217464 40...."


In [16]:
# Sadece gerekli sütunları aldık
lowered_ref = lowered_ref[['mahalle_corrdinate','ilçe','mahalle','geometry_string']]
lowered_ref.head()

,mahalle_corrdinate,ilçe,mahalle,geometry_string
0,"29.055564,40.9087843,29.0555266,40.9089452,29....",adalar,kınalıada,"POLYGON ((29.055564 40.9087843, 29.0555266 40...."
1,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",adalar,burgazada,"POLYGON ((29.0574181 40.8840861, 29.05749 40.8..."
2,"29.0832149,40.8768968,29.082399,40.876799,29.0...",adalar,heybeliada,"POLYGON ((29.0832149 40.8768968, 29.082399 40...."
3,"29.121725,40.8521407,29.1215908,40.8526682,29....",adalar,nizam,"POLYGON ((29.121725 40.8521407, 29.1215908 40...."
4,"29.121725,40.8521407,29.1217464,40.8519378,29....",adalar,maden,"POLYGON ((29.121725 40.8521407, 29.1217464 40...."


In [17]:
# Gereksiz kelimeleri çıkarttık
lowered_df['Mahalle/Köy'] = remove_unwanted_substring(lowered_df['Mahalle/Köy'], ' mah.')
lowered_df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,GBP,CHP,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI
0,1,istanbul,adalar,burgazada,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.318725,56.095618,5.577689,0.079681,0.956175,0.0,0.159363,0.079681,0.717131,0.0
1,1,istanbul,adalar,heybeliada,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.068681,41.311813,10.542582,0.103022,1.476648,0.0,0.171703,0.000000,0.171703,0.0
2,1,istanbul,adalar,kinaliada,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,47.382199,6.020942,0.196335,0.981675,0.0,0.261780,0.000000,0.261780,0.0
3,1,istanbul,adalar,maden,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.087260,42.728330,6.631763,0.116347,1.803374,0.0,0.232693,0.029087,0.349040,0.0
4,1,istanbul,adalar,nizam,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.185048,41.894893,7.809030,0.074019,1.665433,0.0,0.296077,0.074019,0.444115,0.0


In [18]:
# Deneme amaçlı word counter yazdım. Burada mahalle, köy gibi şeyler gözümden kaçtı mı diye kontrol ettim.
from collections import Counter
def count_word_occurrences(df, column_name):
    # Join all the text into one large string
    all_text = ' '.join(df[column_name])
    
    # Split the text into words and count each occurrence
    word_counts = Counter(all_text.split())
    
    return word_counts

In [19]:
map_df= lowered_ref
ysk_df = lowered_df

In [20]:
"""
DENEME
"""
# # Function to match names and return the best match
# def get_best_match(name_to_match, list_of_names):
#     return process.extractOne(name_to_match, list_of_names)[0]
# 
# # Matching districts
# map_df['matched_district'] = map_df['ilçe'].apply(lambda x: get_best_match(x, ysk_df['İlçe Adı'].unique()))
# 
# # Matching neighborhoods within each matched district
# for district in map_df['matched_district'].unique():
#     district_map = map_df[map_df['matched_district'] == district]
#     district_ysk = ysk_df[ysk_df['İlçe Adı'] == district]
#     map_df.loc[map_df['matched_district'] == district, 'matched_neighborhood'] = district_map['mahalle'].apply(
#         lambda x: get_best_match(x, district_ysk['Mahalle/Köy'].unique()))
# 
# # Create a temporary key in both dataframes for merging
# map_df['merge_key'] = map_df['matched_district'] + map_df['matched_neighborhood']
# ysk_df['merge_key'] = ysk_df['İlçe Adı'] + ysk_df['Mahalle/Köy']
# 
# # Left join ysk dataframe with map dataframe
# final_df = pd.merge(ysk_df, map_df, on='merge_key', how='left')

'\nDENEME\n'

In [21]:
#Bu fonksiyon önce ilçeleri kendilerine en benzeyenleri ile eşleştiriyor. Ardından mahalleleri. 
# İnanılmaz bir fonksiyon bu.

import pandas as pd
from fuzzywuzzy import process

def merge_with_fuzzy_matching(map_df, another_df, map_district_col, map_neighborhood_col, another_district_col, another_neighborhood_col):
    # Function to match names and return the best match
    def get_best_match(name_to_match, list_of_names):
        return process.extractOne(name_to_match, list_of_names)[0]

    # Matching districts
    map_df['matched_district'] = map_df[map_district_col].apply(lambda x: get_best_match(x, another_df[another_district_col].unique()))

    # Matching neighborhoods within each matched district
    for district in map_df['matched_district'].unique():
        district_map = map_df[map_df['matched_district'] == district]
        district_another = another_df[another_df[another_district_col] == district]
        map_df.loc[map_df['matched_district'] == district, 'matched_neighborhood'] = district_map[map_neighborhood_col].apply(
            lambda x: get_best_match(x, district_another[another_neighborhood_col].unique()))

    # Create a temporary key in both dataframes for merging
    map_df['merge_key'] = map_df['matched_district'] + map_df['matched_neighborhood']
    another_df['merge_key'] = another_df[another_district_col] + another_df[another_neighborhood_col]

    # Left join another_df with map_df
    final_df = pd.merge(another_df, map_df, on='merge_key', how='left')

    return final_df

In [22]:
merged_df = merge_with_fuzzy_matching(
    map_df, ysk_df, 
    map_district_col='ilçe', map_neighborhood_col='mahalle', 
    another_district_col='İlçe Adı', another_neighborhood_col='Mahalle/Köy'
)

In [23]:
merged_df.head()

,Bölge,İl Adı,İlçe Adı,Mahalle/Köy,Oy Kullanan Seçmen Sayısı,MİLLET,HAK-PAR,TKP,TKH,SOL PARTİ,...,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI,merge_key,mahalle_corrdinate,ilçe,mahalle,geometry_string,matched_district,matched_neighborhood
0,1,istanbul,adalar,burgazada,1255,0.079681,0.000000,0.159363,0.239044,0.398406,...,0.079681,0.717131,0.0,adalarburgazada,"29.0574181,40.8840861,29.05749,40.8836134,29.0...",adalar,burgazada,"POLYGON ((29.0574181 40.8840861, 29.05749 40.8...",adalar,burgazada
1,1,istanbul,adalar,heybeliada,2912,0.171703,0.068681,0.274725,0.034341,0.137363,...,0.000000,0.171703,0.0,adalarheybeliada,"29.0832149,40.8768968,29.082399,40.876799,29.0...",adalar,heybeliada,"POLYGON ((29.0832149 40.8768968, 29.082399 40....",adalar,heybeliada
2,1,istanbul,adalar,kinaliada,1528,0.000000,0.130890,0.196335,0.065445,0.261780,...,0.000000,0.261780,0.0,adalarkinaliada,"29.055564,40.9087843,29.0555266,40.9089452,29....",adalar,kınalıada,"POLYGON ((29.055564 40.9087843, 29.0555266 40....",adalar,kinaliada
3,1,istanbul,adalar,maden,3438,0.029087,0.029087,0.116347,0.116347,0.378127,...,0.029087,0.349040,0.0,adalarmaden,"29.121725,40.8521407,29.1217464,40.8519378,29....",adalar,maden,"POLYGON ((29.121725 40.8521407, 29.1217464 40....",adalar,maden
4,1,istanbul,adalar,nizam,2702,0.074019,0.000000,0.259067,0.111029,0.259067,...,0.074019,0.444115,0.0,adalarnizam,"29.121725,40.8521407,29.1215908,40.8526682,29....",adalar,nizam,"POLYGON ((29.121725 40.8521407, 29.1215908 40....",adalar,nizam


In [29]:
# Tamamen işe yaramıyor her zaman o nedenle burada excel üzerinde elle düzeltmeler yapıyorum.
# Yukarıdaki fonksiyon tarafından eşleştirilemeyen 998 mahallede 28 mahalle var sadece
# Bunların da bir kısmı mahalle diye değil cezaevi diye geçtiği için.
# 18 cezaevi elle çıkartıldı.
# 15 satırda veri uyumsuzlukları giderildi.
# merged_df.to_excel(path+'kontrol3.xlsx') ------------------------------------------- çalıştırıldı

In [65]:
stk = pd.read_excel(path+'istanbul_stk_dataset.xlsx')
stk.tail()

,Plaka,AnaNevisi,WebSitesi,KurumAd,KurumAdres,BaskanAdSoyad,AltNevisString,Error,Latitude,Longitude,City,District,Neighborhood,PartialMatch
23114,34,EĞİTİM ARAŞTIRMA DERNEKLERİ,www.ozenleegitim.com,ZÜMRÜTEVLER BÖLGESİNDE OTURAN 0 18 YAŞ ÇOCUKLA...,ZÜMRÜTEVLER MAH. YABAN LALESİ-SOKAK DISKAPINO:...,DİLARA DEMİRBULAK,Eğitim Alanında Araştırma Yapan Dernekler,NaN,40.946384,29.156498,İstanbul,Maltepe,Zümrütevler,1.0
23115,34,İNSANİ YARDIM DERNEKLERİ,NaN,ZÜMRÜTEVLER EĞİTİM VE KÜLTÜR DERNEĞİ,ZÜMRÜTEVLER MAH. NİL CD. HANDEGÜL SK. NO:63 M...,MUZAFFER AKMAN,İhtiyaç Sahiplerine Yardım,NaN,40.946045,29.149667,İstanbul,Maltepe,Zümrütevler,1.0
23116,34,DİNİ HİZMETLERİN GERÇEKLEŞTİRİLMESİNE YÖNELİK ...,NaN,ZÜMRÜTEVLER MAHALLESİ HALİL İBRAHİM CAMİİ YAPT...,ZÜMRÜTEVLER MAHALLESİ HANDEGÜL SOKAK 28,AHMET ALICI,İbadethane Yaptırma ve Yaşatmayı Amaçlayan Der...,NaN,40.945137,29.148304,İstanbul,Maltepe,Zümrütevler,0.0
23117,34,DİNİ HİZMETLERİN GERÇEKLEŞTİRİLMESİNE YÖNELİK ...,NaN,ZÜMRÜTEVLER MAHALLESİ MEDİNE CAMİİ YAPTIRMA VE...,ZÜMRÜTEVLER SİNEM SOKAK 35,MUHTESİM EŞLEGÜL,İbadethane Yaptırma ve Yaşatmayı Amaçlayan Der...,NaN,40.941374,29.149204,İstanbul,Maltepe,Zümrütevler,0.0
23118,34,DİNİ HİZMETLERİN GERÇEKLEŞTİRİLMESİNE YÖNELİK ...,NaN,ZÜMRÜTEVLER MAHALLESİ NARLI CAMİİ YAPTIRMA VE ...,CANDAN 1,HÜSEYİN OFLAZ,İbadethane Yaptırma ve Yaşatmayı Amaçlayan Der...,API Error: ZERO_RESULTS,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Sonuç olmayan satırları attık 995 tane
stk = stk[stk['Error']!='API Error: ZERO_RESULTS'].reset_index()
# Yanlış şehirleri attık 688 tane
stk = stk[stk['City']=='İstanbul'].reset_index()
stk.tail()
# Gereksiz sütunları dropla


,level_0,index,Plaka,AnaNevisi,WebSitesi,KurumAd,KurumAdres,BaskanAdSoyad,AltNevisString,Error,Latitude,Longitude,City,District,Neighborhood,PartialMatch
21431,22119,23113,34,TOPLUMSAL DEĞERLERİ YAŞATMA DERNEKLERİ,NaN,ZÜMRÜT İLİM YAYMA DERNEĞİ,ŞAHİNTEPE MAH. YUNUS-SOKAK DISKAPINO:56 A--HAC...,METİN ÇİFTÇİ,Toplumsal Değerleri Yaşatma Dernekleri,NaN,41.071813,28.728552,İstanbul,Başakşehir,Şahintepe,1.0
21432,22120,23114,34,EĞİTİM ARAŞTIRMA DERNEKLERİ,www.ozenleegitim.com,ZÜMRÜTEVLER BÖLGESİNDE OTURAN 0 18 YAŞ ÇOCUKLA...,ZÜMRÜTEVLER MAH. YABAN LALESİ-SOKAK DISKAPINO:...,DİLARA DEMİRBULAK,Eğitim Alanında Araştırma Yapan Dernekler,NaN,40.946384,29.156498,İstanbul,Maltepe,Zümrütevler,1.0
21433,22121,23115,34,İNSANİ YARDIM DERNEKLERİ,NaN,ZÜMRÜTEVLER EĞİTİM VE KÜLTÜR DERNEĞİ,ZÜMRÜTEVLER MAH. NİL CD. HANDEGÜL SK. NO:63 M...,MUZAFFER AKMAN,İhtiyaç Sahiplerine Yardım,NaN,40.946045,29.149667,İstanbul,Maltepe,Zümrütevler,1.0
21434,22122,23116,34,DİNİ HİZMETLERİN GERÇEKLEŞTİRİLMESİNE YÖNELİK ...,NaN,ZÜMRÜTEVLER MAHALLESİ HALİL İBRAHİM CAMİİ YAPT...,ZÜMRÜTEVLER MAHALLESİ HANDEGÜL SOKAK 28,AHMET ALICI,İbadethane Yaptırma ve Yaşatmayı Amaçlayan Der...,NaN,40.945137,29.148304,İstanbul,Maltepe,Zümrütevler,0.0
21435,22123,23117,34,DİNİ HİZMETLERİN GERÇEKLEŞTİRİLMESİNE YÖNELİK ...,NaN,ZÜMRÜTEVLER MAHALLESİ MEDİNE CAMİİ YAPTIRMA VE...,ZÜMRÜTEVLER SİNEM SOKAK 35,MUHTESİM EŞLEGÜL,İbadethane Yaptırma ve Yaşatmayı Amaçlayan Der...,NaN,40.941374,29.149204,İstanbul,Maltepe,Zümrütevler,0.0


In [67]:
stk.drop(['level_0','index','Plaka','WebSitesi','BaskanAdSoyad',
          'Error','KurumAd','KurumAdres','AltNevisString',
         'Latitude','Longitude','PartialMatch'],
        axis= 'columns',
        inplace = True)
stk.head()

,AnaNevisi,City,District,Neighborhood
0,İNSANİ YARDIM DERNEKLERİ,İstanbul,Fatih,Hobyar
1,MESLEKİ ve DAYANIŞMA DERNEKLERİ,İstanbul,Ümraniye,Aşağı Dudullu
2,MESLEKİ ve DAYANIŞMA DERNEKLERİ,İstanbul,Şişli,Halaskargazi
3,SPOR ve SPOR İLE İLGİLİ DERNEKLERİ,İstanbul,Fatih,İskenderpaşa
4,İNSANİ YARDIM DERNEKLERİ,İstanbul,Kadıköy,19 Mayıs


In [70]:
# İ'leri I'ya çevirdik
stk_lowered = replace_character(stk)
stk_lowered.head()

,AnaNevisi,City,District,Neighborhood
0,INSANI YARDIM DERNEKLERI,Istanbul,Fatih,Hobyar
1,MESLEKI ve DAYANIŞMA DERNEKLERI,Istanbul,Ümraniye,Aşağı Dudullu
2,MESLEKI ve DAYANIŞMA DERNEKLERI,Istanbul,Şişli,Halaskargazi
3,SPOR ve SPOR ILE ILGILI DERNEKLERI,Istanbul,Fatih,Iskenderpaşa
4,INSANI YARDIM DERNEKLERI,Istanbul,Kadıköy,19 Mayıs


In [71]:
stk_lowered = stk_lowered.apply(lowercase_strings)

stk_lowered.head()

,AnaNevisi,City,District,Neighborhood
0,insani yardim dernekleri,istanbul,fatih,hobyar
1,mesleki ve dayanişma dernekleri,istanbul,ümraniye,aşağı dudullu
2,mesleki ve dayanişma dernekleri,istanbul,şişli,halaskargazi
3,spor ve spor ile ilgili dernekleri,istanbul,fatih,iskenderpaşa
4,insani yardim dernekleri,istanbul,kadıköy,19 mayıs


In [74]:
stk_lowered['AnaNevisi'] = stk_lowered['AnaNevisi'].str.strip()

# Recreating the pivot table after trimming the spaces
corrected_pivot_table = pd.pivot_table(stk_lowered, index=['City', 'District', 'Neighborhood'], 
                                       columns='AnaNevisi', aggfunc=len, fill_value=0)

corrected_pivot_table.head()

AnaNevisi                         bireysel öğreti ve toplumsal gelişim dernekleri  \
City     District Neighborhood                                                      
istanbul adalar   burgazadası                                                   0   
                  büyükada-maden                                                0   
                  büyükada-nizam                                                0   
                  heybeliada                                                    0   
                  kınalıada                                                     0   

AnaNevisi                         dini hizmetlerin gerçekleştirilmesine yönelik faaliyet gösteren dernekler  \
City     District Neighborhood                                                                                
istanbul adalar   burgazadası                                                     2                           
                  büyükada-maden                                                  1                           
                  büyükada-nizam                                                  1                           
                  heybeliada                                                      1                           
                  kınalıada                                                       1                           

AnaNevisi                         diş türkler ile dayanişma dernekleri  \
City     District Neighborhood                                           
istanbul adalar   burgazadası                                        0   
                  büyükada-maden                                     0   
                  büyükada-nizam                                     0   
                  heybeliada                                         0   
                  kınalıada                                          0   

AnaNevisi                         düşünce temelli dernekler  \
City     District Neighborhood                                
istanbul adalar   burgazadası                             0   
                  büyükada-maden                          0   
                  büyükada-nizam                          0   
                  heybeliada                              0   
                  kınalıada                               0   

AnaNevisi                         engelli dernekleri  \
City     District Neighborhood                         
istanbul adalar   burgazadası                      0   
                  büyükada-maden                   0   
                  büyükada-nizam                   0   
                  heybeliada                       0   
                  kınalıada                        0   

AnaNevisi                         eğitim araştirma  dernekleri  \
City     District Neighborhood                                   
istanbul adalar   burgazadası                                0   
                  büyükada-maden                             2   
                  büyükada-nizam                             0   
                  heybeliada                                 0   
                  kınalıada                                  0   

AnaNevisi                         gida, tarim ve hayvancilik alaninda faaliyet gösteren dernekler  \
City     District Neighborhood                                                                      
istanbul adalar   burgazadası                                                     0                 
                  büyükada-maden                                                  0                 
                  büyükada-nizam                                                  0                 
                  heybeliada                                                      0                 
                  kınalıada                                                       0                 

AnaNevisi                         hak ve savunuculuk dernekleri  \
City     District Neighborhood    

In [78]:
flattened_pivot_table = corrected_pivot_table.reset_index()
flattened_pivot_table.to_excel(path+'pivot_kontrol2.xlsx')
flattened_pivot_table.head()


AnaNevisi,City,District,Neighborhood,bireysel öğreti ve toplumsal gelişim dernekleri,dini hizmetlerin gerçekleştirilmesine yönelik faaliyet gösteren dernekler,diş türkler ile dayanişma dernekleri,düşünce temelli dernekler,engelli dernekleri,eğitim araştirma dernekleri,"gida, tarim ve hayvancilik alaninda faaliyet gösteren dernekler",...,"kültür, sanat ve turizm dernekleri",mesleki ve dayanişma dernekleri,sağlik alaninda faaliyet gösteren dernekler,spor ve spor ile ilgili dernekleri,toplumsal değerleri yaşatma dernekleri,uluslar arasi teşekküller ve işbirliği dernekleri,yaşli ve çocuklara yönelik dernekler,çevre doğal hayat hayvanlari koruma dernekleri,çocuk dernekleri,şehit yakini ve gazi dernekleri
0,istanbul,adalar,burgazadası,0,2,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1,istanbul,adalar,büyükada-maden,0,1,0,0,0,2,0,...,0,1,0,0,0,0,0,1,0,0
2,istanbul,adalar,büyükada-nizam,0,1,0,0,0,0,0,...,1,2,0,0,0,0,0,0,0,0
3,istanbul,adalar,heybeliada,0,1,0,0,0,0,0,...,0,2,0,1,0,0,1,0,0,0
4,istanbul,adalar,kınalıada,0,1,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0


In [79]:
def remove_unwanted_substring_if_last(df_column, unwanted_substring):
    """
    Removes occurrences of an unwanted substring from a DataFrame column
    only if the substring is the last word in the string.

    Parameters:
    df_column (pd.Series): The DataFrame column from which to remove the substring.
    unwanted_substring (str): The substring to remove.

    Returns:
    pd.Series: The modified column with the unwanted substring removed if it's the last word.
    """
    # Regular expression: the unwanted substring followed by end-of-string anchor ($)
    regex_pattern = unwanted_substring + '$'
    
    # Use the str accessor with replace to handle substring replacement
    return df_column.str.replace(regex_pattern, '', regex=True)


In [80]:
flattened_pivot_table['mahalle'] = remove_unwanted_substring_if_last(flattened_pivot_table['Neighborhood'], 'mahallesi')
flattened_pivot_table['mahalle'] = remove_unwanted_substring_if_last(flattened_pivot_table['Neighborhood'], 'mah.')
flattened_pivot_table['mahalle'] = remove_unwanted_substring_if_last(flattened_pivot_table['Neighborhood'], 'mh.')
flattened_pivot_table['mahalle'] = remove_unwanted_substring_if_last(flattened_pivot_table['Neighborhood'], 'merkez')
flattened_pivot_table['mahalle'] = remove_unwanted_substring_if_last(flattened_pivot_table['Neighborhood'], 'mh')



flattened_pivot_table.head()

AnaNevisi,City,District,Neighborhood,bireysel öğreti ve toplumsal gelişim dernekleri,dini hizmetlerin gerçekleştirilmesine yönelik faaliyet gösteren dernekler,diş türkler ile dayanişma dernekleri,düşünce temelli dernekler,engelli dernekleri,eğitim araştirma dernekleri,"gida, tarim ve hayvancilik alaninda faaliyet gösteren dernekler",...,mesleki ve dayanişma dernekleri,sağlik alaninda faaliyet gösteren dernekler,spor ve spor ile ilgili dernekleri,toplumsal değerleri yaşatma dernekleri,uluslar arasi teşekküller ve işbirliği dernekleri,yaşli ve çocuklara yönelik dernekler,çevre doğal hayat hayvanlari koruma dernekleri,çocuk dernekleri,şehit yakini ve gazi dernekleri,mahalle
0,istanbul,adalar,burgazadası,0,2,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,burgazadası
1,istanbul,adalar,büyükada-maden,0,1,0,0,0,2,0,...,1,0,0,0,0,0,1,0,0,büyükada-maden
2,istanbul,adalar,büyükada-nizam,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,büyükada-nizam
3,istanbul,adalar,heybeliada,0,1,0,0,0,0,0,...,2,0,1,0,0,1,0,0,0,heybeliada
4,istanbul,adalar,kınalıada,0,1,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,kınalıada


In [57]:
ref_df['İlçe Adı'] = ref_df['İlçe Adı'].fillna('Unknown').astype(str)
ref_df['Mahalle/Köy'] = ref_df['Mahalle/Köy'].fillna('Unknown').astype(str)

flattened_pivot_table['District'] = flattened_pivot_table['District'].fillna('Unknown').astype(str)
flattened_pivot_table['Neighborhood'] = flattened_pivot_table['Neighborhood'].fillna('Unknown').astype(str)

In [58]:
merged_stk_df = merge_with_fuzzy_matching(
    ref_df, stk_lowered, 
    map_district_col='İlçe Adı', map_neighborhood_col='Mahalle/Köy', 
    another_district_col='District', another_neighborhood_col='Neighborhood'
)
merged_stk_df.head()

,AnaNevisi,City,District,Neighborhood,count,merge_key,Unnamed: 0.1,Unnamed: 0,ilçe,mahalle,...,İYİ PARTİ,AP,ZAFER PARTİSİ,SOSYALİST GÜÇ BİRLİĞİ İTTİFAKI,CUMHUR İTTİFAKI,EMEK VE ÖZGÜRLÜK İTTİFAKI,MİLLET İTTİFAKI,ATA İTTİFAKI,mahalle_corrdinate,geometry_string
0,insani yardim dernekleri,istanbul,fatih,hobyar,2.0,fatihhobyar,509.0,530.0,fatih,hobyar,...,11.320755,0.000000,1.886792,0.000000,0.000000,0.000000,0.000000,0.000000,"28.9742404,41.0178689,28.9742604,41.0179128,28...","POLYGON ((28.9742404 41.0178689, 28.9742604 41..."
1,mesleki ve dayanişma dernekleri,istanbul,ümraniye,aşağı dudullu,12.0,ümraniyeaşağı dudullu,239.0,254.0,ümraniye,aşağı dudullu,...,6.881451,0.271087,3.326035,0.010426,0.521322,0.010426,0.323220,0.000000,"29.1596863,41.0227089,29.1596809,41.0225511,29...","POLYGON ((29.1596863 41.0227089, 29.1596809 41..."
2,mesleki ve dayanişma dernekleri,istanbul,şişli,halaskargazi,9.0,şişlihalaskargazi,657.0,681.0,şişli,halaskargazi,...,9.029779,0.192123,1.440922,0.000000,0.096061,0.000000,0.576369,0.000000,"28.9875753,41.0482664,28.9883528,41.0495412,28...","POLYGON ((28.9875753 41.0482664, 28.9883528 41..."
3,spor ve spor ile ilgili dernekleri,istanbul,fatih,iskenderpaşa,4.0,fatihiskenderpaşa,543.0,564.0,fatih,iskenderpaşa,...,7.239007,0.269219,2.108884,0.000000,0.463655,0.000000,0.254263,0.000000,"28.9445969,41.0139944,28.945054,41.0136651,28....","POLYGON ((28.9445969 41.0139944, 28.945054 41...."
4,insani yardim dernekleri,istanbul,kadıköy,19 mayıs,2.0,kadıköy19 mayıs,67.0,68.0,kadıköy,19 mayıs,...,9.583418,0.125991,1.479374,0.000000,0.065027,0.004064,0.365779,0.004064,"29.0800381,40.9755019,29.0802875,40.9753014,29...","POLYGON ((29.0800381 40.9755019, 29.0802875 40..."


In [ ]:
# merged_stk_df.to_excel(path+'stk_kontrol2.xlsx')

In [ ]:
# Buradan sonrası Elle kontrol edildi